In [1]:
import time
import numpy as np
import os
import os.path as op
import pandas as pd
import scipy.optimize as so
import pickle
import scipy as sp
import seaborn as sns
import glob

import matplotlib.pyplot as plt

#from idealObserver2 import io_with_derivations

"""
Created on: May 19, 2023
Latest update: August 7, 2023
Author: @zoeheez



Description: 

This script is a draft for testing different processing and analysis pipline for pupil data.

Update Aug 7, 2023: re-define pupil change as 1-2s after outcome subtracted by 0-1s after outcome (z-scored first)

"""

'\nCreated on: May 19, 2023\nLatest update: August 7, 2023\nAuthor: @zoeheez\n\n\n\nDescription: \n\nThis script is a draft for testing different processing and analysis pipline for pupil data.\n\nUpdate Aug 7, 2023: re-define pupil change as 1-2s after outcome subtracted by 0-1s after outcome (z-scored first)\n\n'

### 0. Load the data

In [2]:
# get the project directory (cbandit)
proj_dir = os.path.normpath(os.getcwd()) #+ os.sep + os.pardir)
print(f'project directory: {proj_dir}')

project directory: /home/wah016/cbandit


In [5]:
pupil_data_holder.keys()

dict_keys(['sub1', 'sub2', 'sub3', 'sub4', 'sub5', 'sub6', 'sub7', 'sub8', 'sub9', 'sub10', 'sub11', 'sub12', 'sub13', 'sub14', 'sub15', 'sub16', 'sub17', 'sub18', 'sub19', 'sub20', 'sub21', 'sub22', 'sub23', 'sub24', 'sub25', 'sub26', 'sub27', 'sub28', 'sub29', 'sub30', 'sub31', 'sub32', 'sub33', 'sub34', 'sub35', 'sub36', 'sub37', 'sub38', 'sub39', 'sub40', 'sub41', 'sub42', 'sub43', 'sub44', 'sub45', 'sub46', 'sub47', 'sub48', 'sub49', 'sub50', 'sub51', 'sub52', 'sub53', 'sub54', 'sub55', 'sub56', 'sub57', 'sub58', 'sub59', 'sub60'])

In [28]:
pupil_example = pupil_data_holder['sub35']['sess6']
pupil_example.events

[{'time': '4241755613', 'msg': 'Introduction'},
 {'time': '4246848506', 'msg': 'trial 0'},
 {'time': '4247947715', 'msg': 'response f rt 1078 type trial forced'},
 {'time': '4249979587', 'msg': 'outcome 50'},
 {'time': '4253027395', 'msg': 'trial 1'},
 {'time': '4253560345', 'msg': 'response j rt 507 type trial forced'},
 {'time': '4255592217', 'msg': 'outcome 66'},
 {'time': '4258641486', 'msg': 'trial 2'},
 {'time': '4259207745', 'msg': 'response j rt 547 type trial forced'},
 {'time': '4261239617', 'msg': 'outcome 30'},
 {'time': '4264287425', 'msg': 'trial 3'},
 {'time': '4264837029', 'msg': 'response f rt 524 type trial forced'},
 {'time': '4266852247', 'msg': 'outcome 50'},
 {'time': '4269916709', 'msg': 'trial 4'},
 {'time': '4271132501', 'msg': 'response f rt 1197 type trial free'},
 {'time': '4273164373', 'msg': 'outcome 10'},
 {'time': '4276210359', 'msg': 'trial 5'},
 {'time': '4276909856', 'msg': 'response j rt 684 type trial free'},
 {'time': '4278941728', 'msg': 'outcome 

In [29]:
pupil_example.data

,time_stamp,time_stamp_rel_s,x_pos,y_pos,diameter,time_stamp_int,time_stamp_rel_s_int,x_pos_int,y_pos_int,diameter_int,track_int
0,4241772267,0.000000,0.0,0.0,0.0,4.885441e+09,643.668765,0.0,0.0,0.0,1.0
1,4241788922,0.016655,0.0,0.0,0.0,4.885441e+09,643.668765,0.0,0.0,0.0,1.0
2,4241805577,0.033310,0.0,0.0,0.0,4.885441e+09,643.668765,0.0,0.0,0.0,1.0
3,4241822231,0.049964,0.0,0.0,0.0,4.885441e+09,643.668765,0.0,0.0,0.0,1.0
4,4241838886,0.066619,0.0,0.0,0.0,4.885441e+09,643.668765,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
38578,4885374413,643.602146,0.0,0.0,0.0,4.885441e+09,643.668765,0.0,0.0,0.0,1.0
38579,4885391068,643.618801,0.0,0.0,0.0,4.885441e+09,643.668765,0.0,0.0,0.0,1.0
38580,4885407723,643.635456,0.0,0.0,0.0,4.885441e+09,643.668765,0.0,0.0,0.0,1.0
38581,4885424377,643.652110,0.0,0.0,0.0,4.885441e+09,643.668765,0.0,0.0,0.0,1.0


In [ ]:
pupil_example.data

In [14]:
pupil_example.epoch_tobii(onsets = [357774713], before=0.1, after=0, variable='diameter_int_z', baseline=[], reject=False, blink_ratio_thd=0.2)

KeyError: 'diameter_int_z'

In [4]:
# import the preprocessed pupil and behavioral data
f_pupil = open(proj_dir+"/pupil_data_preprocessed_all.pkl","rb")
pupil_data_holder = pickle.load(f_pupil)

pupil_keys = pupil_data_holder['sub1']['sess1']
print(f'pupil data \n \
    data type: {type(pupil_data_holder)} \n \
    number of subjects: {len(pupil_data_holder)} \n \
    info included for each session: {pupil_keys}')


path = os.getcwd() + '/data_ind'
csv_files = glob.glob(os.path.join(path, "*.csv"))

# loop over the list of csv files
data_dict = {}
for 
    
    # read the csv file
    df = pd.read_csv(f)
    data_dict[int(f[len(path)+4:-4])] = df
    # print the location and filename
    #print('File Name:', f.split("\\")[-1])
    #print()

# sort the data based on subject number
data_list = []
for i in sorted(data_dict):
        data_list.append(data_dict[i])

nsub = len(data_list)
nsess = 8
print('behavioral data')
print('Done importing behavioral data.')
print('number of subjects: ', nsub)

print('data description:')
print('list name: data_list')
print('list length: ', len(data_list))
print('keys: ', data_list[0].columns)

pupil data 
     data type: <class 'dict'> 
     number of subjects: 60 
     info included for each session: <funcs_preprocess_tobii.pupil object at 0x7f4d98e79910>


NameError: name 'f' is not defined

#### Sketches for testing the example pupil data:

In [ ]:
example_data

#### Preprocessing pupil data

For each subject, each session:
1. get the z-scored pupil diameter for each session, stored in the pupil data
2. for each trial, extract all the time stamps for certain events (start, response, outcome, estim1, estim2). Store the info as a new dataframe (96 trials x # category columns) called `time_stamps`
3. using the time stamps above, epoch the pupil trace for each trial, scored in a new dataframe `pupil_epochs`. Store all subjects and all sessions into dict `pupil_epochs_all`;
4. combine `pupil_epochs` with behavioral data for each session; store the concatenate data into dict `joint_data_all`.

In [44]:
def subtract_baseline(example_epochs, baseline):
    pupil_trace = np.array(example_epochs.to_list())
    return np.array([pupil_trace[n,:] - baseline[n] for n in range(len(baseline))])

def get_mean(example_epochs, timepoints):
    epoch = np.array(example_epochs.to_list())
    #print(epoch.shape)
    return np.mean(epoch[:,timepoints], axis=1)

In [45]:
pupil_epochs

,trial,rt,pupil_baseline_start,pupil_baseline_outcome,pupil_size_pre_response,pupil_size_avg_pre_response,pupil_size_response,pupil_size_avg_response,pupil_size_outcome,pupil_size_avg_outcome,...,pupil_change_pre_response,pupil_change_avg_pre_response,pupil_change_response,pupil_change_avg_response,pupil_change_outcome,pupil_change_avg_outcome,pupil_change_avg_outcome1,pupil_change_avg_outcome2,pupil_change_avg_outcome_nassar,pupil_change_avg_outcome3
0,0,0.873,"[-0.13285487334831725, -0.15435727323074105, -...","[0.07491006887873684, 0.13786000971694218, 0.1...","[0.11437311041501608, 0.10994014858446984, 0.1...",0.279017,"[0.353667051165722, 0.39952404529320773, 0.433...",0.475576,"[0.07491006887873684, 0.13786000971694218, 0.1...",0.143308,...,"[0.2382829492237084, 0.23384998739316215, 0.25...",0.402927,"[0.47757688997441433, 0.5234338841019001, 0.55...",0.599485,"[-0.16825035480462835, -0.105300413966423, -0....",-0.099852,-0.231277,-0.525614,0.135573,-0.294337
1,1,0.503,"[0.14530678606560862, 0.1443430154502174, 0.24...","[0.23190501894722576, 0.17961922205809192, 0.1...","[0.34474857188501473, 0.35575292963603705, 0.3...",0.299246,"[0.3427030080472347, 0.3527936049405821, 0.374...",1.147825,"[0.23190501894722576, 0.17961922205809192, 0.1...",0.349440,...,"[0.11949942966692093, 0.13050378741794325, 0.1...",0.073997,"[0.11745386582914089, 0.1275444627224883, 0.14...",0.922576,"[0.13167717110061047, 0.07939137421147663, 0.0...",0.249212,0.340723,0.132520,0.486649,-0.208202
2,2,0.948,"[0.22792893919942742, 0.15482785060969537, 0.1...","[0.1730485920638111, 0.15684330963330576, 0.18...","[-0.5578585701312097, -0.5368051541620953, -0....",-0.249000,"[-0.043562563000114525, -0.010950897831385292,...",-0.182329,"[0.1730485920638111, 0.15684330963330576, 0.18...",-0.001665,...,"[-0.7465211594523885, -0.7254677434832741, -0....",-0.437663,"[-0.23222515232129334, -0.1996134871525641, -0...",-0.370992,"[-0.009005366728989855, -0.025210649159495196,...",-0.183719,-0.103675,-0.827351,-0.852659,-0.723676
3,3,0.633,"[-0.04115643605480424, -0.011971396588699315, ...","[-0.02837186777614119, 0.004751287527410095, 0...","[0.9398780279838435, 0.9354819044102489, 0.927...",0.780104,"[0.6531173173194468, 0.6612715975477902, 0.646...",0.399992,"[-0.02837186777614119, 0.004751287527410095, 0...",0.392350,...,"[0.8911303729920237, 0.8867342494184292, 0.878...",0.731356,"[0.604369662327627, 0.6125239425559704, 0.5980...",0.351245,"[-0.1277199635170812, -0.09459680821352992, -0...",0.293002,0.652830,-0.133318,0.631348,-0.786147
4,4,1.754,"[0.7752312920948785, 0.8033788108457145, 0.831...","[0.467254551806917, 0.46196730826422516, 0.473...","[0.13627293908167493, 0.0938579818830942, 0.08...",-0.051292,"[-0.1988164400882881, -0.1855046140065632, -0....",0.438572,"[0.467254551806917, 0.46196730826422516, 0.473...",-0.281571,...,"[-0.7203595828075425, -0.7627745400061232, -0....",-0.907925,"[-1.0554489619775056, -1.0421371358957807, -0....",-0.418061,"[0.01424757300069951, 0.008960329458007676, 0....",-0.734578,-1.060431,-1.494615,-0.399348,-0.434184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,91,0.472,"[0.29022167956877815, 0.3297500577255351, 0.33...","[-0.9795138134511088, -0.98944660803868, -0.99...","[0.4595304716940207, 0.494133129262467, 0.5223...",0.456317,"[0.4413219044145404, 0.35941149778196835, 0.32...",-0.422505,"[-0.9795138134511088, -0.98944660803868, -0.99...",-1.541196,...,"[0.1309549849294786, 0.16555764249792487, 0.19...",0.127741,"[0.11274641764999827, 0.030836011017426224, 0....",-0.751080,"[0.029907264502610165, 0.019974469915038928, 0...",-0.531775,0.381590,0.454312,-0.318775,0.072722
92,92,0.472,"[-1.8930057871373005, -1.9205995857737967, -1....","[-1.4544666610148407, -1.4163745588751955, -1....","[-2.322152803407257, -2.3557155008598123, -2.3...",-2.341006,"[-2.237260329138182, -2.121357375739456, -2.14...",-1.424729,"[-1.4544666610148407, -1.4163745588751955, -1....",-1.348714,.

In [46]:
np.array(pupil_epochs['pupil_size_outcome'].to_list()).shape

(96, 187)

In [52]:
# loop through each subject and each session
pupil_epochs_all = {}
joint_data_all = {}

for sub in range(nsub):
    print(f'preprocessing subject {sub+1}...')
    pupil_epochs_all[f'sub{sub+1}'] = {}
    joint_data_all[f'sub{sub+1}'] = {}
    for sess in range(nsess):
        print(f'session {sess+1}...')
        # skip the damaged sessions
        if sub+1 == 24 and sess+1 == 6:
            continue
        if sub+1 == 34 and sess+1 == 8:
            continue
            
        # get the `pupil` object for a single session
        pupil_example = pupil_data_holder[f'sub{sub+1}'][f'sess{sess+1}']
        
        ## STEP 1: z-score the pupil data for each session
        pupil_example.data['diameter_int_z'] = sp.stats.zscore(pupil_example.data['diameter_int'])
        pupil_data_holder[f'sub{sub+1}'][f'sess{sess+1}'].data['diameter_int_z'] = pupil_example.data['diameter_int_z']
        
        ## STEP 2:
        # extract all the time stamps for a certain event (start, response, outcome, estim1, estim2);
        # goal: creat a pd dataframe, each row = a trial, each column = time stamp for a certain event
        event_list = pupil_example.events
        
        # initialize lists for all event types
        start_list = []
        response_list = []
        rt_list = []
        outcome_list = []
        estim1_list = np.ones(96) * np.nan
        estim2_list = np.ones(96) * np.nan
        # loop through each event and put the time stamp to the corresponding event list
        count = -1
        for i in range(len(event_list)):
            if event_list[i]['msg'][0:5] == 'trial':
                count = count+1
                start_list.append(int(event_list[i]['time']))

            if 'response' in event_list[i]['msg']:
                response_list.append(int(event_list[i]['time']))
                # find the digit of the rt (and then divided by 1000)
                #print(event_list[i]['msg'])
                rt = [float(k) for k in event_list[i]['msg'].split() if k.isdigit()]
                if len(rt) == 0:
                    rt_list.append(np.nan)
                    print('invalid trial')
                else:
                    rt_list.append(rt[0]/1000)

            if 'outcome' in event_list[i]['msg']:
                outcome_list.append(int(event_list[i]['time']))

            if 'estim1' in event_list[i]['msg']:
                estim1_list[count] = (int(event_list[i]['time']))

            if 'estim2' in event_list[i]['msg']:
                estim2_list[count] = (int(event_list[i]['time']))
                
        # create a pd dataframe `time_stamps` to store the time stamps
        trial_nums = np.arange(len(start_list))
        time_stamps = pd.DataFrame({'trial': trial_nums, 'start': start_list,\
                                    'response': response_list, 'outcome': outcome_list,
                                   'estim1': estim1_list, 'estim2': estim2_list, 'rt': rt_list})
        
        
        ## STEP 3: epoch the pupil based on the events, stored in dataframe `pupil_epochs`
        pupil_epochs = pd.DataFrame()
        pupil_epochs['trial'] = time_stamps['trial']
        pupil_epochs['rt'] = time_stamps['rt']
        
        
        # UPDATE NEW BASELINE TO BE [-0.1, 0]s before the start of the trial      
        pupil_example.epoch_tobii(onsets = time_stamps['start'], before=0.1, after=0, variable='diameter_int_z', baseline=[], reject=False, blink_ratio_thd=0.2)
        pupil_epochs['pupil_baseline_start'] = pupil_example.epochs.tolist()
        
        pupil_example.epoch_tobii(onsets = time_stamps['outcome'], before=0.1, after=0, variable='diameter_int_z', baseline=[], reject=False, blink_ratio_thd=0.2)
        pupil_epochs['pupil_baseline_outcome'] = pupil_example.epochs.tolist()
        
        
        # pupil response (no baseline)
        # 1. pupil [-0.4, 0]s before choice
        # 2. pupil [-0.1, 2]s after choice
        # 3. pupil [-0.1, 3]s after outcome
        
        # 1. pre-choice
        pupil_example.epoch_tobii(onsets = time_stamps['response'], before=0.4, after=0, variable='diameter_int_z', baseline=[], reject=False, blink_ratio_thd=0.2)
        t_pre_response = pupil_example.epochs_times    
        pupil_epochs['pupil_size_pre_response'] = pupil_example.epochs.tolist()
        pupil_epochs['pupil_size_avg_pre_response'] = np.mean(pupil_example.epochs,axis=1)
        
        # 2. post-choice
        pupil_example.epoch_tobii(onsets = time_stamps['response'], before=0.1, after=2, variable='diameter_int_z', baseline=[], reject=False, blink_ratio_thd=0.2)
        t_response = pupil_example.epochs_times
        pupil_epochs['pupil_size_response'] = pupil_example.epochs.tolist()
        pupil_epochs['pupil_size_avg_response'] = np.mean(pupil_example.epochs,axis=1)
        
        # 3. post-outcome
        pupil_example.epoch_tobii(onsets = time_stamps['outcome'], before=0.1, after=3, variable='diameter_int_z', baseline=[], reject=False, blink_ratio_thd=0.2)
        t_outcome = pupil_example.epochs_times
        pupil_epochs['pupil_size_outcome'] = pupil_example.epochs.tolist()
        pupil_epochs['pupil_size_avg_outcome'] = np.mean(pupil_example.epochs,axis=1)
        
        
        
        # baselined:
        baseline = np.mean(np.array(pupil_epochs['pupil_baseline_start'].to_list()), axis=1)
        pupil_epochs['pupil_baseline_start_avg'] = baseline
        
        baseline2 = np.mean(np.array(pupil_epochs['pupil_baseline_outcome'].to_list()), axis=1)
        pupil_epochs['pupil_baseline_outcome_avg'] = baseline2
        
        pupil_epochs['pupil_change_pre_response'] = subtract_baseline(pupil_epochs['pupil_size_pre_response'], baseline).tolist()
        pupil_epochs['pupil_change_avg_pre_response'] = np.mean(subtract_baseline(pupil_epochs['pupil_size_pre_response'], baseline),axis=1)

        pupil_epochs['pupil_change_response'] = subtract_baseline(pupil_epochs['pupil_size_response'], baseline).tolist()
        pupil_epochs['pupil_change_avg_response'] = np.mean(subtract_baseline(pupil_epochs['pupil_size_response'], baseline),axis=1)

        pupil_epochs['pupil_change_outcome']= subtract_baseline(pupil_epochs['pupil_size_outcome'], baseline2).tolist()
        pupil_epochs['pupil_change_avg_outcome'] = np.mean(subtract_baseline(pupil_epochs['pupil_size_outcome'], baseline2),axis=1)

        
        sub_epoch1 = (t_outcome >= 0) & (t_outcome < 0.5)
        sub_epoch2 = (t_outcome >= 0.5) & (t_outcome < 1.5)
        pupil_epochs['pupil_change_avg_outcome1'] = np.mean(subtract_baseline(pupil_epochs['pupil_change_outcome'], baseline2)[:,sub_epoch1],axis=1)
        pupil_epochs['pupil_change_avg_outcome2'] = np.mean(subtract_baseline(pupil_epochs['pupil_change_outcome'], baseline2)[:,sub_epoch2],axis=1)

        # ADDED Aug 7, 2023: compute the average pupil change like Nassar et al (2012)
        sub_epoch3 = (t_outcome >= 0) & (t_outcome < 1)
        sub_epoch4 = (t_outcome >= 1) & (t_outcome < 2)

        
        sub_epoch1_response = (t_response >= 0) & (t_response < 0.5)
        sub_epoch2_response = (t_response >= 0.5) & (t_response < 1.5)

        sub_epoch3_response = (t_response >= 0) & (t_response < 1)
        sub_epoch4_response = (t_response >= 1) & (t_response < 2)

        # 4. compute the pupil change based on epochs 
        pupil_epochs_all[f'sub{sub+1}'][f'sess{sess+1}'] = pupil_epochs # store the dataframe
        pupil_epochs['pupil_change_avg_outcome_nassar'] = get_mean(pupil_epochs['pupil_size_outcome'],sub_epoch4) \
                                                        - get_mean(pupil_epochs['pupil_size_outcome'],sub_epoch3)
        
        pupil_epochs['pupil_change_avg_outcome3'] = get_mean(pupil_epochs['pupil_size_outcome'],sub_epoch2) \
                                                        - get_mean(pupil_epochs['pupil_size_outcome'],sub_epoch1)
        

        
        pupil_epochs['pupil_change_avg_response_nassar'] = get_mean(pupil_epochs['pupil_size_response'],sub_epoch4_response) \
                                                        - get_mean(pupil_epochs['pupil_size_response'],sub_epoch3_response)
        
        pupil_epochs['pupil_change_avg_response3'] = get_mean(pupil_epochs['pupil_size_response'],sub_epoch2_response) \
                                                        - get_mean(pupil_epochs['pupil_size_response'],sub_epoch1_response)
        

        
        
        
        # TODO: fix the behavioral data first
        # concatenate with behavioral data
        #behavioral_sub = data_list[sub]
        #behavioral_example = behavioral_sub[behavioral_sub['sess']==1]
        #behavioral_example

        #joint_data = pd.concat([pupil_epochs, behavioral_example], axis=1)
        #joint_data.head()
        
        #joint_data_all[f'sub{sub+1}'][f'sess{sess+1}'] = joint_data # store the dataframe
        
        
        
print('Done.')
        

preprocessing subject 1...
session 1...
session 2...
session 3...
session 4...
session 5...
session 6...
session 7...
session 8...
preprocessing subject 2...
session 1...
session 2...
session 3...
session 4...
session 5...
session 6...
session 7...
session 8...
preprocessing subject 3...
session 1...
session 2...
session 3...
session 4...
session 5...
session 6...
session 7...
session 8...
preprocessing subject 4...
session 1...
session 2...
session 3...
session 4...
session 5...
session 6...
session 7...
session 8...
preprocessing subject 5...
session 1...
session 2...
session 3...
session 4...
session 5...
session 6...
session 7...
session 8...
preprocessing subject 6...
session 1...
session 2...
session 3...
session 4...
session 5...
session 6...
session 7...
session 8...
preprocessing subject 7...
session 1...
session 2...
session 3...
session 4...
session 5...
session 6...
session 7...
session 8...
preprocessing subject 8...
session 1...
session 2...
session 3...
session 4...
sess

In [51]:
pupil_epochs[['pupil_change_avg_outcome_nassar', 'pupil_change_avg_outcome3','pupil_change_avg_response_nassar', 'pupil_change_avg_response3']]

,pupil_change_avg_outcome_nassar,pupil_change_avg_outcome3,pupil_change_avg_response_nassar,pupil_change_avg_response3
0,-0.579784,-0.332727,-0.735486,-0.110253
1,-0.017894,0.095363,-0.224063,-0.065612
2,-0.960198,-0.478855,-0.049533,-0.112410
3,-0.205280,-0.497933,0.467959,0.429654
4,-0.004235,0.425821,-1.697433,-0.811345
...,...,...,...,...
91,0.927299,0.444979,-0.722174,-0.771597
92,0.724711,0.286981,-0.775102,-0.532950
93,-0.492806,-0.402934,-0.213278,0.138900
94,0.261742,0.302763,-0.382095,-0.203857



        
        # pupil CHANGE
        # pre-response [-0.4,0]
        pupil_example.epoch_tobii(onsets = time_stamps['response'], before=0.4, after=0, variable='diameter_int_z', baseline=[], reject=False, blink_ratio_thd=0.2)
        t_pre_response = pupil_example.epochs_times
        pupil_epochs['pupil_size_pre_response'] = pupil_example.epochs.tolist()
        pupil_epochs['pupil_size_avg_pre_response'] = np.mean(pupil_example.epochs,axis=1)
        
        
        # around response
        pupil_example.epoch_tobii(onsets = time_stamps['response'], before=0.1, after=2, variable='diameter_int_z', baseline=[-0.1, 0], reject=False, blink_ratio_thd=0.2)
        t_response = pupil_example.epochs_times
        pupil_epochs['pupil_change_response'] = pupil_example.epochs.tolist()
        pupil_epochs['pupil_change_avg_response'] = np.mean(pupil_example.epochs,axis=1)
        
        sub_epoch1 = (t_response >= 0.5) & (t_response < 1)
        sub_epoch2 = (t_response >= 1) & (t_response < 2)
        pupil_epochs['pupil_change_avg_response1'] = np.mean(pupil_example.epochs[:,sub_epoch1],axis=1)
        pupil_epochs['pupil_change_avg_response2'] = np.mean(pupil_example.epochs[:,sub_epoch2],axis=1)


        
        pupil_example.epoch_tobii(onsets = time_stamps['response'], before=0.1, after=0, variable='diameter_int_z', baseline=[], reject=False, blink_ratio_thd=0.2)
        pupil_epochs['pupil_baseline_response'] = pupil_example.epochs.tolist()
        
        # around outcome
        pupil_example.epoch_tobii(onsets = time_stamps['outcome'], before=0.1, after=2.5, variable='diameter_int_z', baseline=[-0.1, 0], reject=False, blink_ratio_thd=0.2)
        t_outcome = pupil_example.epochs_times

        pupil_epochs['pupil_change_outcome'] = pupil_example.epochs.tolist()
        pupil_epochs['pupil_change_avg_outcome'] = np.mean(pupil_example.epochs,axis=1)
        
        sub_epoch1 = (t_outcome >= 0.5) & (t_outcome < 1.5)
        sub_epoch2 = (t_outcome >= 1.5) & (t_outcome < 2.5)
        pupil_epochs['pupil_change_avg_outcome1'] = np.mean(pupil_example.epochs[:,sub_epoch1],axis=1)
        pupil_epochs['pupil_change_avg_outcome2'] = np.mean(pupil_example.epochs[:,sub_epoch2],axis=1)



        pupil_example.epoch_tobii(onsets = time_stamps['outcome'], before=0.1, after=0, variable='diameter_int_z', baseline=[], reject=False, blink_ratio_thd=0.2)
        pupil_epochs['pupil_baseline_outcome'] = pupil_example.epochs.tolist()
        
        # pupil DIAMETER (not baselined)
        # around response
        pupil_example.epoch_tobii(onsets = time_stamps['response'], before=0.1, after=2, variable='diameter_int_z', baseline=[], reject=False, blink_ratio_thd=0.2)
        #t_response = pupil_example.epochs_times
        pupil_epochs['pupil_size_response'] = pupil_example.epochs.tolist()
        pupil_epochs['pupil_size_avg_response'] = np.mean(pupil_example.epochs,axis=1)
        
        # around outcome
        pupil_example.epoch_tobii(onsets = time_stamps['outcome'], before=0.1, after=2.5, variable='diameter_int_z', baseline=[], reject=False, blink_ratio_thd=0.2)
        #t_outcome = pupil_example.epochs_times
        pupil_epochs['pupil_size_outcome'] = pupil_example.epochs.tolist()
        pupil_epochs['pupil_size_avg_outcome'] = np.mean(pupil_example.epochs,axis=1)
        
        pupil_epochs_all[f'sub{sub+1}'][f'sess{sess+1}'] = pupil_epochs # store the dataframe


In [10]:
pupil_example.epochs.shape

(96, 187)

In [53]:

# IMPORTANT: SAVE the dicts
#pupil_epochs_all = {}
#joint_data_all = {}

# create a binary pickle file 
f = open("pupil_epochs_all_new2.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(pupil_epochs_all,f)

# close file
f.close()

# create a binary pickle file 
#f2 = open("joint_data_all.pkl","wb")

# write the python object (dict) to pickle file
#pickle.dump(joint_data_all,f2)

# close file
#f2.close()


In [15]:
### Testing


f_pupil = open(proj_dir+"/pupil_epochs_all_new2.pkl","rb")
pupil_data_holder = pickle.load(f_pupil)

tt = open(proj_dir+"/t_epochs.pkl","rb")
t_epochs = pickle.load(tt)
t_epochs.keys()

EOFError: Ran out of input

In [13]:
len(pupil_data_holder['sub1']['sess1']['pupil_change_outcome'][0])

187

In [43]:
len(pupil_example.epochs.tolist())

96

In [44]:
example_epochs = pupil_epochs_all['sub1']['sess1']
example_epochs

,trial,rt,pupil_baseline_start,pupil_size_pre_response,pupil_size_avg_pre_response,pupil_size_response,pupil_size_avg_response,pupil_size_outcome,pupil_size_avg_outcome,pupil_baseline_avg,pupil_change_pre_response,pupil_change_avg_pre_response,pupil_change_response,pupil_change_avg_response,pupil_change_outcome,pupil_change_avg_outcome
0,0,0.873,"[-0.13285487334831725, -0.15435727323074105, -...","[0.11437311041501608, 0.10994014858446984, 0.1...",0.279017,"[0.353667051165722, 0.39952404529320773, 0.433...",0.475576,"[0.07491006887873684, 0.13786000971694218, 0.1...",0.143308,-0.123910,"[0.2382829492237084, 0.23384998739316215, 0.25...",0.402927,"[0.47757688997441433, 0.5234338841019001, 0.55...",0.599485,"[0.1988199076874292, 0.2617698485256345, 0.310...",0.267218
1,1,0.503,"[0.14530678606560862, 0.1443430154502174, 0.24...","[0.34474857188501473, 0.35575292963603705, 0.3...",0.299246,"[0.3427030080472347, 0.3527936049405821, 0.374...",1.147825,"[0.23190501894722576, 0.17961922205809192, 0.1...",0.349440,0.225249,"[0.11949942966692093, 0.13050378741794325, 0.1...",0.073997,"[0.11745386582914089, 0.1275444627224883, 0.14...",0.922576,"[0.006655876729131954, -0.04562992016000189, -...",0.124191
2,2,0.948,"[0.22792893919942742, 0.15482785060969537, 0.1...","[-0.5578585701312097, -0.5368051541620953, -0....",-0.249000,"[-0.043562563000114525, -0.010950897831385292,...",-0.182329,"[0.1730485920638111, 0.15684330963330576, 0.18...",-0.001665,0.188663,"[-0.7465211594523885, -0.7254677434832741, -0....",-0.437663,"[-0.23222515232129334, -0.1996134871525641, -0...",-0.370992,"[-0.015613997257367729, -0.03181927968787307, ...",-0.190328
3,3,0.633,"[-0.04115643605480424, -0.011971396588699315, ...","[0.9398780279838435, 0.9354819044102489, 0.927...",0.780104,"[0.6531173173194468, 0.6612715975477902, 0.646...",0.399992,"[-0.02837186777614119, 0.004751287527410095, 0...",0.392350,0.048748,"[0.8911303729920237, 0.8867342494184292, 0.878...",0.731356,"[0.604369662327627, 0.6125239425559704, 0.5980...",0.351245,"[-0.07711952276796097, -0.043996367464409686, ...",0.343602
4,4,1.754,"[0.7752312920948785, 0.8033788108457145, 0.831...","[0.13627293908167493, 0.0938579818830942, 0.08...",-0.051292,"[-0.1988164400882881, -0.1855046140065632, -0....",0.438572,"[0.467254551806917, 0.46196730826422516, 0.473...",-0.281571,0.856633,"[-0.7203595828075425, -0.7627745400061232, -0....",-0.907925,"[-1.0554489619775056, -1.0421371358957807, -0....",-0.418061,"[-0.38937797008230046, -0.3946652136249923, -0...",-1.138204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,91,0.472,"[0.29022167956877815, 0.3297500577255351, 0.33...","[0.4595304716940207, 0.494133129262467, 0.5223...",0.456317,"[0.4413219044145404, 0.35941149778196835, 0.32...",-0.422505,"[-0.9795138134511088, -0.98944660803868, -0.99...",-1.541196,0.328575,"[0.1309549849294786, 0.16555764249792487, 0.19...",0.127741,"[0.11274641764999827, 0.030836011017426224, 0....",-0.751080,"[-1.3080893002156508, -1.3180220948032222, -1....",-1.869771
92,92,0.472,"[-1.8930057871373005, -1.9205995857737967, -1....","[-2.322152803407257, -2.3557155008598123, -2.3...",-2.341006,"[-2.237260329138182, -2.121357375739456, -2.14...",-1.424729,"[-1.4544666610148407, -1.4163745588751955, -1....",-1.348714,-1.962096,"[-0.3600564691907342, -0.39361916664328933, -0...",-0.378910,"[-0.2751639949216591, -0.1592610415229332, -0....",0.537367,"[0.5076296732016823, 0.5457217753413275, 0.592...",0.613382
93,93,0.755,"[-0.2599642376506139, -0.31802739435540434, -0...","[-0.5222524479376774, -0.5479006194615563, -0....",-0.365448,"[-0.2193996432680427, -0.18022596530957216, -0...",-0.081562,"[-1.1383605034071906, -1.1432341543872064, -1....",-0.703429,-0.414443,"[-0.10780985087909939, -0.13345802240297827, -...",0.048995,"[0.19504295379053535, 0.2342166317490059, 0.24...",0.332881,"[-0.7239179063486125, -0.7287915573286283, -0....",-0.288986
94,94,0.422,"[-0.6079342681157723, -0.6871213053228692, -0....","[

In [24]:
event = 'pre_response'
pupil_trace = np.array(example_epochs[f'pupil_size_{event}'].to_list())
pupil_trace

array([[ 1.14373110e-01,  1.09940149e-01,  1.26141264e-01, ...,
         5.09891045e-01,  5.14486709e-01,  5.61021108e-01],
       [ 3.44748572e-01,  3.55752930e-01,  3.47091768e-01, ...,
         3.40205419e-01,  3.79542730e-01,  4.33235533e-01],
       [-5.57858570e-01, -5.36805154e-01, -5.71089712e-01, ...,
         9.22384549e-03,  1.23313902e-03,  4.35450208e-02],
       ...,
       [-5.22252448e-01, -5.47900619e-01, -5.46774559e-01, ...,
        -6.38564126e-02, -4.82106925e-02, -3.66055779e-02],
       [-1.18526821e+00, -1.29686964e+00, -1.37299382e+00, ...,
        -1.42287291e+00, -1.42868930e+00, -1.40255738e+00],
       [-2.70153876e+00, -2.67205232e+00, -2.65676160e+00, ...,
        -2.12687589e+00, -2.03908551e+00, -2.01577771e+00]])

In [25]:
baseline = np.mean(np.array(example_epochs[f'pupil_baseline_start'].to_list()), axis=1)
baseline.shape

(96,)

In [31]:
np.array([pupil_trace[n,:] - baseline[n] for n in range(len(baseline))])

array([[ 0.23828295,  0.23384999,  0.2500511 , ...,  0.63380088,
         0.63839655,  0.68493095],
       [ 0.11949943,  0.13050379,  0.12184263, ...,  0.11495628,
         0.15429359,  0.20798639],
       [-0.74652116, -0.72546774, -0.7597523 , ..., -0.17943874,
        -0.18742945, -0.14511757],
       ...,
       [-0.10780985, -0.13345802, -0.13233196, ...,  0.35058618,
         0.3662319 ,  0.37783702],
       [-0.3834489 , -0.49505033, -0.57117451, ..., -0.6210536 ,
        -0.62686999, -0.60073807],
       [ 0.02240267,  0.05188911,  0.06717983, ...,  0.59706555,
         0.68485593,  0.70816373]])

In [33]:
for s in range()

array([[ 1.98819908e-01,  2.61769849e-01,  3.10332664e-01, ...,
         2.66335733e-01,  3.09281402e-01,  2.69216625e-01],
       [ 6.65587673e-03, -4.56299202e-02, -8.34147345e-02, ...,
         7.52564379e-02,  2.67979698e-03, -7.04212916e-02],
       [-1.56139973e-02, -3.18192797e-02, -6.98019896e-04, ...,
        -2.70691826e-01, -2.73354551e-01, -2.29819025e-01],
       ...,
       [-7.23917906e-01, -7.28791557e-01, -7.60503901e-01, ...,
        -1.37034238e-01, -1.93491671e-01, -2.72678708e-01],
       [-1.39445083e+00, -1.39221513e+00, -1.43067745e+00, ...,
        -1.84500955e+00, -1.86127942e+00, -1.89596396e+00],
       [ 1.45550008e+00,  1.47262939e+00,  1.53620753e+00, ...,
         1.36500102e+00,  1.38168974e+00,  1.36945729e+00]])

In [30]:
res = [int(i) for i in event_list[2]['msg'].split() if i.isdigit()]
type(res[0])

int

In [27]:
event_list[2]['msg'].split()

['response', 'j', 'rt', '784', 'type', 'trial', 'forced']

In [40]:
## next step: preprocess behavioral data

#

In [41]:
time_stamps

,trial,start,response,outcome,estim1,estim2,rt
0,1,8955791294,8956590719,8958622591,NaN,NaN,0.784
1,2,8961670399,8962236658,8964251875,NaN,NaN,0.538
2,3,8967316338,8967915907,8969931124,NaN,NaN,0.582
3,4,8972995586,8973695083,8975710300,NaN,NaN,0.677
4,5,8978758108,8979174475,8981206347,NaN,NaN,0.391
...,...,...,...,...,...,...,...
91,92,9528028931,9529194759,9531209976,NaN,NaN,1.142
92,93,9534257784,9534590878,9536606095,NaN,NaN,0.316
93,94,9539653903,9540086925,9542102142,NaN,NaN,0.412
94,95,9545149950,9545483043,9547498260,NaN,NaN,0.319


In [45]:
t_pre_response

array([-0.399713, -0.383058, -0.366403, -0.349749, -0.333094, -0.316439,
       -0.299785, -0.28313 , -0.266475, -0.249821, -0.233166, -0.216511,
       -0.199857, -0.183202, -0.166547, -0.149892, -0.133238, -0.116583,
       -0.099928, -0.083274, -0.066619, -0.049964, -0.03331 , -0.016655,
        0.      ])

In [54]:
t_epochs = {'t_pre_response': t_pre_response, 't_response': t_response, 't_outcome': t_outcome}
t_epochs

{'t_pre_response': array([-0.399713, -0.383058, -0.366403, -0.349749, -0.333094, -0.316439,
        -0.299785, -0.28313 , -0.266475, -0.249821, -0.233166, -0.216511,
        -0.199857, -0.183202, -0.166547, -0.149892, -0.133238, -0.116583,
        -0.099928, -0.083274, -0.066619, -0.049964, -0.03331 , -0.016655,
         0.      ]),
 't_response': array([-0.099928, -0.083274, -0.066619, -0.049964, -0.03331 , -0.016655,
         0.      ,  0.016654,  0.033309,  0.049964,  0.066618,  0.083273,
         0.099928,  0.116583,  0.133237,  0.149892,  0.166547,  0.183201,
         0.199856,  0.216511,  0.233165,  0.24982 ,  0.266475,  0.283129,
         0.299784,  0.316439,  0.333093,  0.349748,  0.366403,  0.383058,
         0.399712,  0.416367,  0.433022,  0.449676,  0.466331,  0.482986,
         0.49964 ,  0.516295,  0.53295 ,  0.549604,  0.566259,  0.582914,
         0.599568,  0.616223,  0.632878,  0.649533,  0.666187,  0.682842,
         0.699497,  0.716151,  0.732806,  0.749461,  0.7661

In [55]:
# create a binary pickle file 
f3 = open("t_epochs.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(t_epochs,f3)

# close file
f3.close()